In [99]:
import numpy as np
import heapq
import math
import time
import os

In [100]:
def computeMinPath(graph, numVertices):
  minPath = np.zeros((numVertices, 2))
  for i in range(numVertices):
    edgeWeights = graph[i]
    
    minEdge = np.min(edgeWeights)
    secondMinEdge = np.partition(edgeWeights, 1)[1]

    minPath[i] = [minEdge, secondMinEdge]

  return minPath
      
def getOnePath(graph, numVertices):
  path = np.array([0])
  cost = 0
  for i in range(1, numVertices):
    cost += graph[path[-1]][i]
    path = np.concatenate([path, [i]]).astype(int)
  cost += graph[path[-1]][0]
  path = np.concatenate([path, [0]]).astype(int)
  return path, cost

def bound(graph, minPath, nextVertice, prevNode=None):
  if prevNode is None:
    return np.sum(minPath) / 2
  
  cost = graph[prevNode.path[-1]][nextVertice]

  bound = prevNode.bound * 2

  if cost >= minPath[nextVertice][1]:
    bound -= minPath[nextVertice][1]
    bound += cost
  
  if cost >= minPath[prevNode.path[-1]][1]:
    bound -= minPath[prevNode.path[-1]][1]
    bound += cost

  return bound / 2


## Branch and Bound

In [101]:
class Node:
  def __init__(self, graph, minPath, prevNode=None, nextVertice=None):
    if prevNode is None or nextVertice is None:
      self.level = 1
      self.path = [0]
      self.cost = 0
      self.bound = bound(graph, minPath, 0)
      return

    self.level = prevNode.level + 1
    self.path = np.concatenate([prevNode.path, [nextVertice]]).astype(int)
    if len(prevNode.path) == 0:
      self.cost = 0
    else:
      self.cost = prevNode.cost + graph[prevNode.path[-1]][nextVertice]
    self.bound = bound(graph, minPath, nextVertice, prevNode)

  def __lt__(self, other):
    return self.bound < other.bound
  
  def __gt__(self, other):
    return self.bound > other.bound
  
  def __eq__(self, other):
    return self.bound == other.bound
  
  def __str__(self) -> str:
    return f"Path: {self.path} Cost: {self.cost} Bound: {self.bound}"

In [102]:
def branchAndBoundTSP(graph, numVertices, timeLimit=1800):
  startTime = time.time()

  minPath = computeMinPath(graph, numVertices)

  root = Node(graph, minPath, None, 0)
  queue = [root]
  heapq.heapify(queue)

  solution, best = getOnePath(graph, numVertices)

  while queue and time.time() - startTime < timeLimit:
    node = heapq.heappop(queue)
    
    if node.level >= numVertices:
      finalNode = Node(graph, minPath, node, 0)
      if finalNode.cost < best:
        best = finalNode.cost
        solution = finalNode.path
    
    elif node.bound < best:
      unvisited = np.setdiff1d(np.arange(numVertices), node.path)
      for k in unvisited:
        newNode = Node(graph, minPath, node, k)
        if newNode.bound < best and graph[node.path[-1]][k] != np.inf:
          heapq.heappush(queue, newNode)

  return solution, best

## Twice Around the Tree

In [103]:
def generateMST(graph, numVertices):
  included = [False] * numVertices
  included[0] = True

  cost = 0
  edges = []
  numVisited = 1

  while numVisited < numVertices:
    minEdge = (float('inf'), None, None) 

    for i in range(numVertices):
      if included[i]:
        for j in range(numVertices):
          if not included[j] and graph[i][j] < minEdge[0]:
            minEdge = (graph[i][j], i, j)

    cost, startVertex, endVertex = minEdge
    included[endVertex] = True
    cost += cost
    edges.append((startVertex, endVertex))
    numVisited += 1

  mstAdjMatrix = np.full((numVertices, numVertices), np.inf)
  for edge in edges:
    mstAdjMatrix[edge[0]][edge[1]] = mstAdjMatrix[edge[1]][edge[0]] = graph[edge[0]][edge[1]]

  return mstAdjMatrix, cost

In [104]:
def preorderWalkAux(adjMatrix, startVertex, visited, order):
  order.append(startVertex)
  visited[startVertex] = True

  for neighbor in range(len(adjMatrix)):
    if adjMatrix[startVertex][neighbor] != np.inf and not visited[neighbor]:
      preorderWalkAux(adjMatrix, neighbor, visited, order)

def preorderWalk(adjMatrix, start_vertex=0):
  numVertices = len(adjMatrix)
  visited = [False] * numVertices
  order = []

  preorderWalkAux(adjMatrix, start_vertex, visited, order)

  return order

In [105]:
def approximateTwiceAroundTheTree(graph, numVertices):
  mstAdjMatrix, mstCost = generateMST(graph, numVertices)

  preorder = preorderWalk(mstAdjMatrix)
  tour = preorder + [preorder[0]]

  tourCost = sum(graph[tour[i]][tour[i + 1]] for i in range(len(tour) - 1))

  return tour, tourCost

## Christofides

In [106]:
def findMinimumWeightMatching(graph, oddVertices):
  if oddVertices is None:
    oddVertices = range(len(graph))

  edges = []
  for i in oddVertices:
    for j in oddVertices:
      if i < j and graph[i][j] != np.inf:
        edges.append((i, j, graph[i][j]))

  edges.sort(key=lambda x: x[2])

  matching = []
  visited = np.zeros(len(graph), dtype=bool)

  for edge in edges:
    if not visited[edge[0]] and not visited[edge[1]]:
      matching.append((edge[0], edge[1]))
      visited[edge[0]] = True
      visited[edge[1]] = True

  return matching

In [107]:
def getEulerianCircuit(graph):
  startVertex = 0  # Choose a starting vertex
  numVertices = len(graph)
  visited = np.zeros(numVertices, dtype=bool)
  tour = []
  stack = np.array([startVertex])

  while stack.size > 0:
    vertex = stack[-1]
    if not visited[vertex]:
      visited[vertex] = True
      tour.append(vertex)
    unvisited_neighbors = np.where((graph[vertex] != np.inf) & (~visited))[0]
    if unvisited_neighbors.size > 0:
      next_vertex = unvisited_neighbors[0]
      stack = np.append(stack, next_vertex)
    else:
      stack = stack[:-1]

  return tour

In [108]:
def christofides(graph, numVertices):
  mstAdjMatrix, mstCost = generateMST(graph, numVertices)

  oddVertices = [vertex for vertex, degree in enumerate(np.sum(mstAdjMatrix != np.inf, axis=1)) if degree % 2 != 0]

  matchingEdges = findMinimumWeightMatching(graph, oddVertices)

  multigraph = mstAdjMatrix.copy()
  for edge in matchingEdges:
    multigraph[edge[0]][edge[1]] = multigraph[edge[1]][edge[0]] = graph[edge[0]][edge[1]]

  eulerianCircuit = getEulerianCircuit(multigraph)
  hamiltonianCircuit = list(dict.fromkeys(eulerianCircuit))
  hamiltonianCircuit.append(hamiltonianCircuit[0])

  tourCost = sum(graph[hamiltonianCircuit[i]][hamiltonianCircuit[i + 1]] for i in range(len(hamiltonianCircuit) - 1))

  return hamiltonianCircuit, tourCost

## Reading input files

In [109]:
def readCoordinatesFromFile(filename):
  coordinates = []

  with open(filename, 'r') as file:
    for line in file:
      if "NODE_COORD_SECTION" in line:
        break

    for line in file:
      if line.strip() == "EOF":
        break
      id, x, y = map(float, line.split())
      coordinates.append((int(id)-1, x, y))

  return coordinates

def calculateEuclideanDistance(point1, point2):
    x1, y1 = point1[1], point1[2]
    x2, y2 = point2[1], point2[2]
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def getGraphFromFile(filename):
    coordinates = readCoordinatesFromFile(filename)
    numPoints = len(coordinates)

    graph = np.full((numPoints, numPoints), np.inf)

    for i in range(numPoints):
      for j in range(i+1, numPoints):
        distance = calculateEuclideanDistance(coordinates[i], coordinates[j])
        graph[coordinates[i][0]][coordinates[j][0]] = graph[coordinates[j][0]][coordinates[i][0]] = distance

    return graph

In [110]:
def getFilesInFolder(folderPath):
  return [os.path.join(folderPath, file) for file in os.listdir(folderPath) if os.path.isfile(os.path.join(folderPath, file))]

files = getFilesInFolder("data")

## Testing

In [111]:
numVertices = 4
graph = np.full((numVertices, numVertices), np.inf)
graph = np.array([
       [0, 10, 15, 20],
       [10, 0, 35, 25],
       [15, 35, 0, 30],
       [20, 25, 30, 0]
       ])
solution, best = branchAndBoundTSP(graph, numVertices)

print(f"Solution: {solution} Cost: {best}")

Solution: [0 1 3 2 0] Cost: 80


In [112]:
outputFolder = "out"
os.makedirs(outputFolder, exist_ok=True)

for file in files:
  graph = getGraphFromFile(file)
  numVertices = len(graph)

  fileNameWithoutExtension = os.path.splitext(os.path.basename(file))[0]
  outputFilename = os.path.join(outputFolder, f"results_{fileNameWithoutExtension}.txt")

  with open(outputFilename, "w") as outputFile:
    outputFile.write(f"---------- Dataset: {file} - Number of vertices: {numVertices} ----------\n")

    # Branch and Bound
    start_time = time.time()
    solution, best = branchAndBoundTSP(graph, numVertices, 4)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Branch and Bound:")
    print(f"Cost: {best}")
    print(f"Solution: {solution}")
    print(f"Elapsed Time: {elapsed_time} seconds\n")
    outputFile.write("Branch and Bound:\n")
    outputFile.write(f"Cost: {best}\n")
    outputFile.write(f"Solution: {solution}\n")
    outputFile.write(f"Elapsed Time: {elapsed_time} seconds\n\n")

    # Twice Around The Tree
    start_time = time.time()
    solution, best = approximateTwiceAroundTheTree(graph, numVertices)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Twice Around the Tree:")
    print(f"Cost: {best}")
    print(f"Solution: {solution}")
    print(f"Elapsed Time: {elapsed_time} seconds\n")
    outputFile.write("Twice Around the Tree:\n")
    outputFile.write(f"Cost: {best}\n")
    outputFile.write(f"Solution: {solution}\n")
    outputFile.write(f"Elapsed Time: {elapsed_time} seconds\n\n")

    # Christofides
    start_time = time.time()
    solution, best = christofides(graph, numVertices)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Christofides:")
    print(f"Cost: {best}")
    print(f"Solution: {solution}")
    print(f"Elapsed Time: {elapsed_time} seconds\n")
    outputFile.write("Christofides:\n")
    outputFile.write(f"Cost: {best}\n")
    outputFile.write(f"Solution: {solution}\n")
    outputFile.write(f"Elapsed Time: {elapsed_time} seconds\n\n")

    print("----------------------------------------\n")
    outputFile.write("----------------------------------------\n")

  print(f"Results for {file} saved to {outputFilename}\n")


Branch and Bound:
Cost: 3231697.3391183307
Solution: [   0    1    2 ... 1302 1303    0]
Elapsed Time: 4.214607000350952 seconds

Twice Around the Tree:
Cost: 375888.93422715313
Solution: [0, 382, 326, 555, 554, 553, 552, 551, 550, 549, 80, 548, 547, 279, 546, 545, 544, 543, 542, 541, 536, 376, 1214, 861, 375, 862, 535, 224, 237, 190, 189, 150, 164, 188, 187, 186, 158, 157, 156, 155, 1254, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 653, 1253, 232, 231, 230, 225, 238, 200, 201, 202, 119, 355, 239, 226, 196, 650, 716, 717, 120, 718, 719, 720, 721, 722, 723, 724, 725, 165, 170, 171, 172, 173, 174, 151, 175, 176, 177, 1196, 208, 207, 206, 205, 204, 203, 209, 210, 354, 353, 352, 351, 350, 349, 348, 347, 346, 345, 249, 344, 343, 342, 341, 340, 339, 338, 337, 23, 336, 335, 334, 715, 714, 713, 593, 24, 1027, 568, 432, 469, 811, 847, 846, 512, 332, 331, 36, 333, 845, 436, 37, 844, 511, 843, 842, 96, 841, 840, 789, 794, 839, 838, 837, 462, 836, 835, 834, 833,

KeyboardInterrupt: 